## What is Index Intersection?

Index intersection provides the ability to execute a performant query that spans multiple indexes in your data store. This means you can write ad-hoc, dynamically generated queries, where you don't need to know the query syntax or ordering of fields in advance.

## Why should I care?

In MongoDB (or any database), in order to execute a covered query (one that is most efficient) that spans multiple fields, you need to create a [compound index](https://www.mongodb.com/docs/manual/core/index-compound/). Then when you issue your query, it has to match the ordering of the fields in your compound index. 

Take this document:

``` json
{
  "title": "Fight Club",
  "year": 1999,
  "imdb": {
    "rating": 8.9,
    "votes": 1191784,
    "id": 137523
  },
  "cast":[
    "Edward Norton",
    "Brad Pitt"
  ]
}
```

Let's say I have a movie search application and I know what my users are going to look for in advance. They care most about the `title` and `year` fields when they find movies. So I create a compound index like:

`db.products.createIndex( { "title": 1, "year": 1 } )`

Then my query looks like:

`db.collection.find({"title":"Fight Club", "year":1999})`

If we then run an `.explain()` plan, we know it's an `IXSCAN`:

``` json
{
  "queryPlanner": {
    "parsedQuery": {
      "$and": [{
        "title": {
          "$eq": "Fight Club"
        }
      }, {
        "year": {
          "$eq": 1999
        }
      }]
    },
    "winningPlan": {
      "stage": "FETCH",
      "inputStage": {
        "stage": "IXSCAN",
        "keyPattern": {
          "title": 1,
          "year": 1
        },
        "indexBounds": {
          "title": ["[\"Fight Club\", \"Fight Club\"]"],
          "year": ["[1999, 1999]"]
        }
      }
    },
  }
}
```

Now if we add to the query, say we also want to filter by `{"cast":"Edward Norton"}` it will result in a Collection Scan, which should be avoided at all costs. 

## Alternatively: Atlas Search

We can issue all of parameters together within a must (AND) clause, which is the equivalent of  `db.collection.find({"title":"Fight Club", "year":1999})`:

```
[{
  "$search": {
    "compound": {
      "must": [{
          "text": {
            "query": "Fight Club",
            "path": "title"
          }
        },
        {
          "range": {
            "path": "year",
            "gte": 1999,
            "lte": 1999
          }
        }
      ]
    }
  }
}]
```

Then when we add `cast` to the query, we can still get performant results:

```
[{
  "$search": {
    "compound": {
      "must": [{
          "text": {
            "query": "Fight Club",
            "path": "title"
          }
        },
        {
          "range": {
            "path": "year",
            "gte": 1999,
            "lte": 1999
          }
        },
        {
          "text": {
            "query": "Edward Norton",
            "path": "cast"
          }
        }
      ]
    }
  }
}]
```



### TODO
- Explain plan output
- Use actual code/db executions
- 